In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df=pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")
# df.head()
df.info()
df.memory_usage().sum()/pow(2,10)


# Data preparation and summary

Clean data
* remove rows where `daily_vaccinations` are NaN

In [ ]:
print( df.shape)
original_rows = df.shape[0]
# df[pd.isnull(df.daily_vaccinations)]
df = df.dropna(subset=['daily_vaccinations'])
deleted_rows =  original_rows - df.shape[0]
print( "deleted " + str(deleted_rows/original_rows*100)+"% data" )

## List of vaccines

In [ ]:
def flat_unique_elements(array_of_array):
    vaccine_list = []
    for childArray in array_of_array:
        for child in childArray.split(","):
            if child.strip() not in vaccine_list:
                vaccine_list.append(child.strip())
    return vaccine_list

vaccines = flat_unique_elements(df['vaccines'].unique())

print("list of all vaccines  -\n", vaccines)

## List of vaccines and countries who have started its vaccination

     vaccine    country
     
     vaccine 1  country 1, 3
     vaccine 2  country 1, 4

In [ ]:
vaccine_country_list = pd.DataFrame(columns = ['vaccines','country'])
df_vaccine_country = df[["country", "vaccines"]].drop_duplicates()
# for vaccine in vaccines:
for vaccine in vaccines:
    c = df_vaccine_country[df_vaccine_country.vaccines.str.contains(vaccine)]
    d = df_vaccine_country[df_vaccine_country.vaccines.map(lambda p : vaccine in p)]
    a = flat_unique_elements(c.country)
    s = ",".join(x for x in a)
    vaccine_country_list = vaccine_country_list.append({'vaccines': vaccine, 'country': s}, ignore_index=True)

print(vaccine_country_list)

## Find out which vaccines are used by a country

    country     vaccines

    country 1   vaccine 1, 2, 3
    country 1   vaccine 1, 2, 3

In [ ]:
country_vaccines = df[['country','vaccines']].drop_duplicates()
all_country = df['country'].unique()

country_vaccine = pd.DataFrame(columns = ['country','vaccines'])

if(all_country.size==country_vaccines.shape[0]):
    country_vaccine = country_vaccines
else:
    for country in all_country:
        vacciness= country_vaccines[ country_vaccines.country == country ]
        a= ",".join( str(x) for x in flat_unique_elements(vacciness.vaccines) )
        country_vaccine = country_vaccine.append({'country': country , 'vaccines': str(a) }, ignore_index=True)

print(country_vaccine)

# Task 1

##  What vaccines are used and in which countries?

In [ ]:
vaccine_country_list


## Q2. What country is vaccinated more people?

In [ ]:
most_vaccinated_country = df.loc[df.total_vaccinations.idxmax()]

In [ ]:
most_vaccinated_country

# Visualization

## Plot indias vaccine progress

In [ ]:

required_columns = ['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'vaccines']

indian_vaccine_data= df.loc[df['country']=="India",required_columns]
print(indian_vaccine_data.head())
# indian_vaccine_data.plot( indian_vaccine_data['date'], indian_vaccine_data['total_vaccinations'])
indian_vaccine_data.plot()

##  Number of days of vaccination campaingn

In [ ]:
country_days_of_vaccination =df.country.where(df.people_vaccinated>0).value_counts()
country_days_of_vaccination.plot()